## Serving of Tensorflow2 model with Konduit-serving

Step 1: Initiate the server with  
konduit serve --config config.yaml  

Step 2: Run client.ipynb

In [4]:
import re
import io
import logging
import time
import numpy as np
import pickle
import tensorflow as tf

from konduit.load import client_from_file

logging.basicConfig(level='DEBUG')
logging.info("Test")


INFO:root:Test


In [5]:

input_path = 'D:\\Users\\chiawei\\konduit\\Github\\newsgroup_data\\20news-bydate\\20news-bydate-test\\misc.forsale\\76128'
#input_path = 'D:\\Users\\chiawei\\konduit\\Github\\newsgroup_data\\20news-bydate\\20news-bydate-test\\rec.autos\\103066'
#input_path = 'D:\\Users\\chiawei\\konduit\\Github\\newsgroup_data\\20news-bydate\\20news-bydate-test\\comp.os.ms-windows.misc\\10003'

handle = open(input_path, 'r')
input_data = list([handle.read()])

tokenizer_path = 'tokenizer.pickle'
handle = open(tokenizer_path, 'rb')
tokenizer = pickle.load(handle)

print('Load index label')
label_path = "labelclass.pickle"
labelhandler = open(label_path, 'rb')
labelhandler = pickle.load(labelhandler)

MAX_LEN = 256
input_sequence = tokenizer.texts_to_sequences(input_data)
input = tf.keras.preprocessing.sequence.pad_sequences(input_sequence, maxlen = MAX_LEN, padding = "post")

client = client_from_file("config.yaml")


DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:65322


Load index label


DEBUG:urllib3.connectionpool:http://localhost:65322 "GET /healthcheck HTTP/1.1" 204 0
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:65322
DEBUG:urllib3.connectionpool:http://localhost:65322 "GET /config HTTP/1.1" 200 910
INFO:root:Retrieved config is


In [8]:
responses = []

start = time.time()

for i in range(10):
    response = client.predict({"default": input})
    responses.append(response)

end = time.time()

print("%f seconds elapsed for %d requests (%d RPS)" % (end - start, len(responses), (10.0 / (end - start))))

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:65322
DEBUG:urllib3.connectionpool:http://localhost:65322 "POST /classification/numpy HTTP/1.1" 200 610
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:65322
DEBUG:urllib3.connectionpool:http://localhost:65322 "POST /classification/numpy HTTP/1.1" 200 610
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:65322
DEBUG:urllib3.connectionpool:http://localhost:65322 "POST /classification/numpy HTTP/1.1" 200 610
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:65322
DEBUG:urllib3.connectionpool:http://localhost:65322 "POST /classification/numpy HTTP/1.1" 200 610
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:65322
DEBUG:urllib3.connectionpool:http://localhost:65322 "POST /classification/numpy HTTP/1.1" 200 610
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:65322
DEBUG:urllib3.connectionpool:http://

20.536259 seconds elapsed for 10 requests (0 RPS)


In [9]:
response = client.predict({"default": input})

results = response["output"]["probabilities"]
index = int(np.argmax(response['output']['probabilities'], 1)[0])

print("Class: {}".format(labelhandler[index]))
print("Probabilities: {}".format(np.max(response['output']['probabilities'])))

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:65322
DEBUG:urllib3.connectionpool:http://localhost:65322 "POST /classification/numpy HTTP/1.1" 200 610


Class: misc.forsale
Probabilities: 0.9295534491539001
